In [37]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.data import (DataLoader, Dataset)

import pickle
import sys
sys.path.append("../")

import torch
from torch.nn.utils.rnn import PackedSequence
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.nn as nn

# from transformers import BertTokenizer, BertForQuestionAnswering, BertConfig, AutoModel

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients
from src.module.BaselineModule import BaselineModule
from src.data.SeqData import SeqData


from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients, IntegratedGradients


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [135]:
from sklearn.decomposition import PCA

X = np.random.randn(1024, 768)
pca = PCA(n_components=300)
pca.fit(X)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PCA(n_components=300)

In [136]:
X.shape

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(1024, 768)

In [129]:
pca.components_.shape

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(300, 768)

In [137]:
vec = X[0]
u = pca.components_[0]

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


(300,)

In [139]:
np.dot(u.T, vec)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.093087305678251

In [140]:
u @ vec

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.093087305678251

In [43]:
from torch.nn.utils.rnn import *


raw = [ torch.ones(25, 300) / 2, 
        torch.ones(22, 300) / 2.3, 
        torch.ones(15, 300) / 3.2 ]
padded = pad_sequence(raw)  # size: [25, 3, 300]


lengths = torch.as_tensor([25, 22, 15], dtype=torch.int64)
packed = pack_padded_sequence(padded, lengths)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [46]:
p2 = pack_sequence(raw, enforce_sorted=False)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [50]:
p2.batch_sizes

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1,
        1])

In [42]:
packed.data.shape

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


torch.Size([62, 300])

In [ ]:
packed_se

In [51]:
# 
k_fold = 0
CKPT_PATH = "/home/jiashu/seq/logs/save/565/ckpt/epoch00-AUROC0.75-acc0.00.ckpt"
# TwoHeadLSTM
model = BaselineModule.load_from_checkpoint(CKPT_PATH).model
model = model
model.eval()
model.zero_grad()

DATA_PATH = f"../processed/k{k_fold}/train_processed,pkl"
with open(DATA_PATH, "rb") as f:
    seq = pickle.load(f)
sd = SeqData(seq)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/torchmetrics/utilities/prints.py:37: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [84]:
dataloader = DataLoader(
        sd, batch_size=1,
        collate_fn=SeqData.collate_fn, drop_last=False,
        num_workers=1, pin_memory=True
)
batch = next(iter(dataloader))

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [111]:
def forward(L_seq, R_seq, L_len, R_len):
    print(L_seq.shape, L_len.shape)
    print(L_seq.T.shape)
    L = pack_padded_sequence(
        L_seq.T, L_len, enforce_sorted=False
    )
    R = pack_padded_sequence(
        R_seq.T, R_len, enforce_sorted=False
    )
    L = {"gesture": L}
    R = {"gesture": R}
    # (N=1, )
    logits = model(L, R)
    # pseudo-prob
    return logits.sigmoid()

# L_pack = batch["L"]["gesture"]
# R_pack = batch["R"]["gesture"]
# L_seq, L_len = pad_packed_sequence(L_pack)
# R_seq, R_len = pad_packed_sequence(R_pack)
# forward(
#     L_seq, R_seq, L_len, R_len
# )

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [112]:
lig = LayerIntegratedGradients(forward, model.embeddings)

L_pack = batch["L"]["gesture"]
R_pack = batch["R"]["gesture"]
L_seq, L_len = pad_packed_sequence(L_pack)
L_seq = L_seq.view(1, -1)
R_seq = R_seq.view(1, -1)
R_seq, R_len = pad_packed_sequence(R_pack)
attributions, delta= lig.attribute(
    inputs=(L_seq, R_seq),
    additional_forward_args=(L_len, R_len),
    baselines=(torch.zeros_like(L_seq), torch.zeros_like(R_seq)),
    return_convergence_delta=True)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


torch.Size([1, 110]) torch.Size([1])
torch.Size([110, 1])


RuntimeError: start (1) + length (1) exceeds dimension size (1).

In [86]:
def calc_baseline(val):
    return torch.zeros_like(val.data)

/home/jiashu/.conda/envs/meta/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [194]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# IMDB

# BERT

In [195]:
model_path = "csarron/bert-base-uncased-squad-v1"

model = BertForQuestionAnswering.from_pretrained(model_path)
model.to(device)
model.eval()
model.zero_grad()

# load tokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

In [196]:

def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    output = model(inputs, token_type_ids=token_type_ids,
                 position_ids=position_ids, attention_mask=attention_mask, )
    return output.start_logits, output.end_logits

In [197]:
def squad_pos_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
    pred = predict(inputs,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    pred = pred[position]
    return pred.max(1).values

In [198]:

ref_token_id = tokenizer.pad_token_id # A token used for generating token reference
sep_token_id = tokenizer.sep_token_id # A token used as a separator between question and text and it is also added to the end of the text.
cls_token_id = tokenizer.cls_token_id # A token used for prepending to the concatenated question-text word sequence

In [199]:
def construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id):
    question_ids = tokenizer.encode(question, add_special_tokens=False)
    text_ids = tokenizer.encode(text, add_special_tokens=False)

    # construct input token ids
    input_ids = [cls_token_id] + question_ids + [sep_token_id] + text_ids + [sep_token_id]

    # construct reference token ids 
    ref_input_ids = [cls_token_id] + [ref_token_id] * len(question_ids) + [sep_token_id] + \
        [ref_token_id] * len(text_ids) + [sep_token_id]

    return torch.tensor([input_ids], device=device), torch.tensor([ref_input_ids], device=device), len(question_ids)

def construct_input_ref_token_type_pair(input_ids, sep_ind=0):
    seq_len = input_ids.size(1)
    token_type_ids = torch.tensor([[0 if i <= sep_ind else 1 for i in range(seq_len)]], device=device)
    ref_token_type_ids = torch.zeros_like(token_type_ids, device=device)# * -1
    return token_type_ids, ref_token_type_ids

def construct_input_ref_pos_id_pair(input_ids):
    seq_length = input_ids.size(1)
    position_ids = torch.arange(seq_length, dtype=torch.long, device=device)
    # we could potentially also use random permutation with `torch.randperm(seq_length, device=device)`
    ref_position_ids = torch.zeros(seq_length, dtype=torch.long, device=device)

    position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
    ref_position_ids = ref_position_ids.unsqueeze(0).expand_as(input_ids)
    return position_ids, ref_position_ids
    
def construct_attention_mask(input_ids):
    return torch.ones_like(input_ids)

def construct_whole_bert_embeddings(input_ids, ref_input_ids, \
                                    token_type_ids=None, ref_token_type_ids=None, \
                                    position_ids=None, ref_position_ids=None):
    input_embeddings = model.bert.embeddings(input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    ref_input_embeddings = model.bert.embeddings(ref_input_ids, token_type_ids=token_type_ids, position_ids=position_ids)
    
    return input_embeddings, ref_input_embeddings


In [200]:
question, text = "What is important to us?", "It is important to us to include, empower and support humans of all kinds."


In [201]:
# (1, 26) [cls] q [seq] a [seq]
# (1, 26) [cls] pad... [seq] pad.. [seq]
# #question tokens = sep_id
input_ids, ref_input_ids, sep_id = construct_input_ref_pair(question, text, ref_token_id, sep_token_id, cls_token_id)
# (1, 26) [0 0 ... 0 1 ... 1] q => 0, a => 1
# (1, 26) [0 ... 0]
token_type_ids, ref_token_type_ids = construct_input_ref_token_type_pair(input_ids, sep_id)
# (1, 26) [0, 1, ... 25]
# (1, 26) [0....0]
position_ids, ref_position_ids = construct_input_ref_pos_id_pair(input_ids)
# (1, 26) all 0
attention_mask = construct_attention_mask(input_ids)

indices = input_ids[0].detach().tolist()
all_tokens = tokenizer.convert_ids_to_tokens(indices)

In [202]:

ground_truth = 'to include, empower and support humans of all kinds'

ground_truth_tokens = tokenizer.encode(ground_truth, add_special_tokens=False)
ground_truth_end_ind = indices.index(ground_truth_tokens[-1])
ground_truth_start_ind = ground_truth_end_ind - len(ground_truth_tokens) + 1


In [203]:
len(ground_truth_tokens)

11

In [204]:
ground_truth_end_ind

23

In [205]:
ground_truth_tokens

[2000, 2421, 1010, 7861, 11452, 1998, 2490, 4286, 1997, 2035, 7957]

In [206]:
# (1, 26) score for each token as start pos
start_scores, end_scores = predict(input_ids, \
                                   token_type_ids=token_type_ids, \
                                   position_ids=position_ids, \
                                   attention_mask=attention_mask)

print('Question: ', question)
print('Predicted Answer: ', ' '.join(all_tokens[torch.argmax(start_scores) : torch.argmax(end_scores)+1]))

Question:  What is important to us?
Predicted Answer:  to include , em ##power and support humans of all kinds


In [207]:
start_scores.shape

torch.Size([1, 26])

In [208]:
# based on pos = 0, start pos 
lig = LayerIntegratedGradients(squad_pos_forward_func, model.bert.embeddings)

# (1, 26, 768)
attributions_start, delta_start = lig.attribute(inputs=input_ids,
                                  baselines=ref_input_ids,
                                  additional_forward_args=(token_type_ids, position_ids, attention_mask, 0),
                                  return_convergence_delta=True)
attributions_end, delta_end = lig.attribute(inputs=input_ids, baselines=ref_input_ids,
                                additional_forward_args=(token_type_ids, position_ids, attention_mask, 1),
                                return_convergence_delta=True)

In [209]:
def summarize_attributions(attributions):
    attributions = attributions.sum(dim=-1).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    # (26, )
    return attributions
attributions_start_sum = summarize_attributions(attributions_start)
attributions_end_sum = summarize_attributions(attributions_end)

In [210]:
# storing couple samples in an array for visualization purposes
start_position_vis = viz.VisualizationDataRecord(
                        attributions_start_sum,
                        torch.max(torch.softmax(start_scores[0], dim=0)),
                        torch.argmax(start_scores),
                        torch.argmax(start_scores),
                        str(ground_truth_start_ind),
                        attributions_start_sum.sum(),       
                        all_tokens,
                        delta_start)

end_position_vis = viz.VisualizationDataRecord(
                        attributions_end_sum,
                        torch.max(torch.softmax(end_scores[0], dim=0)),
                        torch.argmax(end_scores),
                        torch.argmax(end_scores),
                        str(ground_truth_end_ind),
                        attributions_end_sum.sum(),       
                        all_tokens,
                        delta_end)

print('\033[1m', 'Visualizations For Start Position', '\033[0m')
viz.visualize_text([start_position_vis])

print('\033[1m', 'Visualizations For End Position', '\033[0m')
viz.visualize_text([end_position_vis])
print()

 Visualizations For Start Position 


 Visualizations For End Position 


# SENTENCE 

In [318]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder

from transformers import AutoModel, AutoTokenizer, AutoModelForSequenceClassification

from captum.attr import visualization as viz
from captum.attr import LayerConductance, LayerIntegratedGradients

In [319]:
BEST_CKPT = "/home/jiashu/uncanny_valley/motif/huggingface/outs/ckpt/20/checkpoint-950"
model = AutoModelForSequenceClassification.from_pretrained(BEST_CKPT)
model.eval()
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

In [381]:
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

In [387]:
x = tokenizer(sent)

len(x["input_ids"])

1979

In [ ]:
import pandas as pd
atu = pd.read_hdf("/home/jiashu/uncanny_valley/datasets/ATU.h5", key="default")
# df = df[df["motif"] != "NOT FOUND"]
# atu = df.loc[df.groupby("atu")["atu"].filter(lambda g: len(g) >= 10).index]

In [ ]:
all_tokens = tokenizer.convert_ids_to_tokens(
    x["input_ids"][0]
)

In [ ]:
# only when same seq_len
# output = model.roberta(**x)
# emb = output["last_hidden_state"].mean(-1).detach().cpu().numpy()
# np.isclose(emb, case.iloc[0]["roberta-large"]).all()

In [ ]:
x["input_ids"].shape

torch.Size([1, 1979])

In [ ]:
def forward(input_ids, attention_mask, pos: int):
    output = model(input_ids, attention_mask=attention_mask)
    # (N, K)
    logits = output.logits
    # (N, )
    return logits.softmax(1)[:, pos]
forward(x["input_ids"], x["attention_mask"], label)

RuntimeError: The expanded size of the tensor (1979) must match the existing size (514) at non-singleton dimension 1.  Target sizes: [1, 1979].  Tensor sizes: [1, 514]

In [ ]:
x["input_ids"]
# <s> pad pad .. </s>
# OR TokenReferenceBase(tokenizer.pad_token_id)
baseline = torch.tensor([tokenizer.bos_token_id] + [tokenizer.pad_token_id] * (128 - 2) + [tokenizer.eos_token_id]).view(x["input_ids"].shape)

In [ ]:
lig = LayerIntegratedGradients(forward, model.roberta.embeddings)

attributions, delta= lig.attribute(
    inputs=x["input_ids"],
    baselines=baseline,
    additional_forward_args=(x["attention_mask"], label),
    internal_batch_size=5,
    return_convergence_delta=True)

In [293]:
def summarize(attr):
    # attr (1, seq_len, hidden)
    # (seq_len)
    attr = attr.sum(dim=-1).squeeze(0)
    return attr / torch.norm(attr)
summary = summarize(attributions)

In [294]:
logits = model(**x).logits
prob = logits.softmax(-1)
y_pred = prob.argmax(-1)
print(f"sent: {sent[:129]}")
print(f"label: {label}, {case.iloc[0]['motif']}")
print(f"predict: {y_pred.item()}")

sent: There was once a young man who had wandered out into the world to seek his fortune. As he went his way he met an old man who aske
label: 6, Tales of Magic
predict: 6


In [295]:
y_pred

tensor([6])

In [296]:
score_vis = viz.VisualizationDataRecord(
    word_attributions=summary,
    pred_prob = prob[0, label].item(),
    pred_class=y_pred.item(),
    true_class=case.iloc[0]['motif'],
    attr_class=y_pred.item(),
    # sent[:129],
    attr_score=summary.sum(),
    raw_input=list(map(lambda s: s.replace("Ġ", ""), all_tokens)),
    convergence_score=delta)
print('\033[1m', 'Visualization For Score', '\033[0m')
viz.visualize_text([score_vis])
print()

 Visualization For Score 
